# Lab 2.5: Custom Inference Loop

**Objective**: Build custom inference using AutoModel and AutoTokenizer

**Duration**: 25 minutes

## Learning Outcomes
- Load models with AutoModel
- Tokenize inputs manually
- Run forward pass and decode outputs

In [ ]:
import sys
sys.path.insert(0, "../../../src")
from hf_ecosystem import __version__
print(f"hf-ecosystem version: {__version__}")

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from hf_ecosystem.data import tokenize_batch

## 1. Loading Model and Tokenizer

In [ ]:
model_id = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

print(f"Model: {model.config.model_type}")
print(f"Labels: {model.config.id2label}")

## 2. Manual Tokenization

In [ ]:
texts = ["I love this!", "This is terrible."]

# Tokenize
inputs = tokenize_batch(texts, tokenizer, max_length=64)
print(f"Input IDs shape: {inputs['input_ids'].shape}")

## 3. Forward Pass

In [ ]:
# Run inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions
probs = torch.softmax(outputs.logits, dim=-1)
predictions = torch.argmax(probs, dim=-1)

for text, pred, prob in zip(texts, predictions, probs):
    label = model.config.id2label[pred.item()]
    confidence = prob[pred].item()
    print(f"{text} → {label} ({confidence:.3f})")

## Verification

In [ ]:
def verify_lab():
    assert predictions[0].item() == 1  # POSITIVE
    assert predictions[1].item() == 0  # NEGATIVE
    print("✅ Lab completed successfully!")

verify_lab()